# Naive Bayes text classification

Naive Bayes model with TF/IDF algorithm to solve text classification problem


In [51]:
import re
import csv
import math
from collections import Counter
from collections import defaultdict
import numpy as np
import nltk

In [52]:
demo_data = [["Chinese Beijing Chinese","0"],
             ["Chinese Chinese Shanghai","0"],
             ["Chinese Macao","0"],
             ["Tokyo Japan Chinese","1"]]
demo_pred =  "Chinese Chinese Chinese Tokyo Japan"

In [53]:
class NaiveBayes:
    def __init__(self, data, isUseLog = False):
        self.corpus, self.labels  = self.data_split(data)
        self.classes = Counter(self.labels)
        self.isUseLog = isUseLog
     
    def data_split(self, data):
        corpus = []
        labels = []
        for text, label in data:
            corpus.append(text.lower().split())
            labels.append(label)
        return corpus, labels
    
    def get_data_statistic(self):
        print ("*** NaiveBayes data statistic ***")
        print ("Corpus length = ", len(self.corpus))
        print ("classes count = ", dict(self.classes))
        print ("classes ratio = ", {i: self.classes[i]/len(self.corpus) for i in self.classes} )
        print ("unique words in corpus = ", self.get_unique_words())
        print ("*** ------------------------- ***")
    
    def get_unique_words(self):
        unique_words_in_corpus = Counter()
        for doc in self.corpus:
            unique_words_in_corpus += Counter(doc)
        return len(unique_words_in_corpus)
    
    def get_class_words(self):
        total = defaultdict(int)
        words = defaultdict(Counter)       
        for i in range(len(self.corpus)):           
            words[self.labels[i]] += Counter(self.corpus[i])
            total[self.labels[i]] += len(self.corpus[i])
        return words, total
    
    def get_prior (self, class_label):
        return self.classes[class_label]/len(self.corpus)          
    
    def get_p (self, class_label, word):
        # P(word|class) = (word_count_in_class + 1)/(total_words_in_class+total_unique_words_in_corpus) 
        p = (self.words[class_label][word] + 1)/(self.total[class_label] + self.unique)
        return p
    
    def fit(self):
        self.words, self.total = self.get_class_words()
        self.unique = self.get_unique_words()
            
    def predict_doc(self, doc):
        p = dict()
        for label in self.classes:
            p[label] = self.get_prior (label)
            if self.isUseLog:
                p[label] = math.log(p[label])
            for word in doc.lower().split():
                if self.isUseLog:
                    p[label] += math.log(self.get_p(label,word))
                else:
                    p[label] *= self.get_p(label,word)
        y = max(p, key=p.get)
        return y, p 
    
    def predict(self, docs):
        matches = []
        for doc, label in docs:
            y,p = self.predict_doc(doc)
            matches.append(y == label)
        return Counter(matches)
            

In [54]:
# Test NaiveBayes Model for demo data

# pobability
nbm = NaiveBayes(demo_data)
nbm.get_data_statistic()
nbm.fit()
y,p = nbm.predict_doc(demo_pred)
print("pobability   ", y, p)

# log
nbm = NaiveBayes(demo_data, True)
nbm.fit()
y,p = nbm.predict_doc(demo_pred)
print("log          ", y, p)

# Must return[ ('Chinese Chinese Chinese Tokyo Japan', '0')]
# pobability {'1': 0.00013548070246744226, '0': 0.00030121377997263036}
# or log     {'1': -7.906681345001262, '0': -7.10769031284391}

*** NaiveBayes data statistic ***
Corpus length =  4
classes count =  {'0': 3, '1': 1}
classes ratio =  {'0': 0.75, '1': 0.25}
unique words in corpus =  6
*** ------------------------- ***
pobability    0 {'0': 0.00030121377997263036, '1': 0.00013548070246744226}
log           0 {'0': -8.10769031284391, '1': -8.906681345001262}


In [55]:
%%time

# data.csv prepare dataset

def read_csv_file(file_name):
    with open(file_name, 'r') as csv_file:
        reader = csv.reader(csv_file)
        data = [doc for doc in reader]
        csv_file.close()    
    return data

def delete_stopwords(data):
    # nltk.download('stopwords')  # 1 time or nltk.download()
    stopwords = nltk.corpus.stopwords.words('english')
    pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
    for i in range(len(data)):   
        data[i][0] = data[i][0].lower()
        data[i][0] = pattern.sub('', data[i][0])   # filter(lambda x: x not in stopwords, data[i][0])
    return data

def train_verif_split(data, train_persent):
    train_count = int(len(data)*train_persent/100.0)
    train = data[:train_count]
    verif = data[train_count:]
    return train, verif

# print statistic
data = read_csv_file("data.csv")
train, verif = train_verif_split (data, 80.0)

print(len(data), "=", len(train), "+", len(verif))
print("------------")

nbm = NaiveBayes(data)
nbm.get_data_statistic()

nbm = NaiveBayes(train)
nbm.get_data_statistic()

nbm = NaiveBayes(verif)
nbm.get_data_statistic()

1118 = 894 + 224
------------
*** NaiveBayes data statistic ***
Corpus length =  1118
classes count =  {'0': 380, '1': 738}
classes ratio =  {'0': 0.33989266547406083, '1': 0.6601073345259392}
unique words in corpus =  33697
*** ------------------------- ***
*** NaiveBayes data statistic ***
Corpus length =  894
classes count =  {'0': 297, '1': 597}
classes ratio =  {'0': 0.33221476510067116, '1': 0.6677852348993288}
unique words in corpus =  26275
*** ------------------------- ***
*** NaiveBayes data statistic ***
Corpus length =  224
classes count =  {'1': 141, '0': 83}
classes ratio =  {'1': 0.6294642857142857, '0': 0.3705357142857143}
unique words in corpus =  15295
*** ------------------------- ***
Wall time: 1.81 s


In [56]:
%%time

# data.csv model
data = read_csv_file("data.csv")
train, verif = train_verif_split (data, 80.0)

nbm = NaiveBayes(train)
nbm.fit()
matches = nbm.predict(verif)
print ("pobability")
print ("Matches:  ", matches) 
print ("Accuracy: ", {i: matches[i]/len(verif) for i in matches}) 
print ("----------")

nbm = NaiveBayes(train, True)
nbm.fit()
matches = nbm.predict(verif)
print ("log")
print ("Matches:  ", matches) 
print ("Accuracy: ", {i: matches[i]/len(verif) for i in matches}) 
print ("----------")

nbm = NaiveBayes(delete_stopwords(train), True)
nbm.fit()
matches = nbm.predict(delete_stopwords(verif))
print ("log without stopwords")
print ("Matches:  ", matches) 
print ("Accuracy: ", {i: matches[i]/len(verif) for i in matches}) 
print ("----------")


pobability
Matches:   Counter({True: 117, False: 107})
Accuracy:  {False: 0.47767857142857145, True: 0.5223214285714286}
----------
log
Matches:   Counter({True: 214, False: 10})
Accuracy:  {True: 0.9553571428571429, False: 0.044642857142857144}
----------
log without stopwords
Matches:   Counter({True: 211, False: 13})
Accuracy:  {True: 0.9419642857142857, False: 0.05803571428571429}
----------
Wall time: 4.17 s


# TF-IDF algorithm

## Term Frequency
TF — это частотность термина, которая измеряет, насколько часто термин встречается в документе. Логично предположить, что в длинных документах термин может встретиться в больших количествах, чем в коротких, поэтому абсолютные числа тут не катят. Поэтому применяют относительные — делят количество раз, когда нужный термин встретился в тексте, на общее количество слов в тексте. 

## Inverse Document Frequency
IDF — это обратная частотность документов. Она измеряет непосредственно важность термина. То есть, когда мы считали TF, все термины считаются как бы равными по важности друг другу. Но всем известно, что, например, предлоги встречаются очень часто, хотя практически не влияют на смысл текста. И что с этим поделать? Ответ прост — посчитать IDF. Он считается как логарифм от общего количества документов, делённого на количество документов, в которых встречается термин а.

#### TF термина а = (Количество раз, когда термин а встретился в тексте / количество всех слов в тексте)
#### IDF термина а = (Общее количество документов / Количество документов, в которых встречается термин а)

In [59]:
# TF-IDF

nbm = NaiveBayes(demo_data)
print(nbm.corpus)

def calc_tf(doc):
    """
    parameters: doc - List of words
    returns: Counter object with TF for all words in docum
    """
    tf = Counter(doc)
    for i in tf:
        tf[i] = tf[i]/float(len(doc))
    return tf

def calc_idf(word, corpus):
    """
    parameters: corpus - List of docs
                word - or which to calculate IDF
    returns: value, idf for word in corpus            
    """
    word_in_doc_count = sum([1.0 for i in corpus if word in i])
    idf = len(corpus) / word_in_doc_count
    return idf # math.log(idf)
      
def calc_tfidf(corpus):
    docs_list = []
    for doc in corpus:
        tf_idf_dict = {}
        tf = calc_tf(doc)
        for word in tf:
            tf_idf_dict[word] = tf[word] / calc_idf(word, corpus)
        docs_list.append(tf_idf_dict)
    return docs_list

texts = [['pasta', 'la', 'vista', 'baby', 'la', 'vista'], 
         ['hasta', 'siempre', 'comandante', 'baby', 'la', 'siempre'], 
         ['siempre', 'comandante', 'baby', 'la', 'siempre']]
print (calc_tfidf(texts))
print ("---------------")
print (calc_tfidf(nbm.corpus))



[['chinese', 'beijing', 'chinese'], ['chinese', 'chinese', 'shanghai'], ['chinese', 'macao'], ['tokyo', 'japan', 'chinese']]
[{'pasta': 0.05555555555555555, 'la': 0.3333333333333333, 'vista': 0.1111111111111111, 'baby': 0.16666666666666666}, {'hasta': 0.05555555555555555, 'siempre': 0.2222222222222222, 'comandante': 0.1111111111111111, 'baby': 0.16666666666666666, 'la': 0.16666666666666666}, {'siempre': 0.26666666666666666, 'comandante': 0.13333333333333333, 'baby': 0.2, 'la': 0.2}]
---------------
[{'chinese': 0.6666666666666666, 'beijing': 0.08333333333333333}, {'chinese': 0.6666666666666666, 'shanghai': 0.08333333333333333}, {'chinese': 0.5, 'macao': 0.125}, {'tokyo': 0.08333333333333333, 'japan': 0.08333333333333333, 'chinese': 0.3333333333333333}]


- http://nlpx.net/archives/57
- https://stevenloria.com/tf-idf/
- https://ru.wikipedia.org/wiki/TF-IDF